In [1]:
import sys


sys.path.append("../src/")

In [2]:
from sennet.core.mmap_arrays import read_mmap_array, create_mmap_array
import matplotlib.pyplot as plt
import numpy as np

In [3]:
import cc3d

In [4]:
# labels_out = cc3d.connected_components(labels_in) # 26-connected
# labels_out = cc3d.dust(
#     labels_in, 
#     threshold=100, 
#     connectivity=26, 
#     in_place=False,
# )

In [5]:
from pathlib import Path


# folder = "kidney_3_dense"
folder = "kidney_2"
root_path = Path(f"/home/clay/research/kaggle/sennet/data_dumps/predicted/ensembled/{folder}/")
mmap_paths = sorted([p for p in root_path.glob("chunk_*")])
for p in mmap_paths:
    print(p)
mmaps = [read_mmap_array(p / "thresholded_prob")for p in mmap_paths]
print([m.shape for m in mmaps])
mmap = np.concatenate([m.data for m in mmaps], axis=0)
# mmap = read_mmap_array("/home/clay/research/kaggle/sennet/data_dumps/predicted/ensembled/kidney_3_dense/chunk_02/thresholded_prob")

/home/clay/research/kaggle/sennet/data_dumps/predicted/ensembled/kidney_2/chunk_00
/home/clay/research/kaggle/sennet/data_dumps/predicted/ensembled/kidney_2/chunk_01
/home/clay/research/kaggle/sennet/data_dumps/predicted/ensembled/kidney_2/chunk_02
/home/clay/research/kaggle/sennet/data_dumps/predicted/ensembled/kidney_2/chunk_03
/home/clay/research/kaggle/sennet/data_dumps/predicted/ensembled/kidney_2/chunk_04
[[444, 1041, 1511], [444, 1041, 1511], [444, 1041, 1511], [444, 1041, 1511], [441, 1041, 1511]]


In [6]:
connectivity = 26 # only 4,8 (2D) and 26, 18, and 6 (3D) are allowed
labels_out, num_labels = cc3d.connected_components(
    mmap, 
    connectivity=connectivity, 
    # out_file="out.bin",
    return_N=True,
)
labels_out = cc3d.dust(
    labels_out, 
    threshold=1000,
    connectivity=26, 
    in_place=False,
)

In [7]:
!ls -lah

total 269M
drwxrwxr-x  3 clay clay 4.0K Dec 25 16:51 .
drwxrwxr-x 14 clay clay 4.0K Dec 25 13:24 ..
-rw-rw-r--  1 clay clay  36K Dec 19 23:38 2d_mask_proc.ipynb
-rw-rw-r--  1 clay clay  15K Dec 25 16:51 exp_cc3d.ipynb
-rw-rw-r--  1 clay clay 666K Dec 23 18:05 exp_clahe.ipynb
-rw-rw-r--  1 clay clay  72K Dec 22 21:12 exp_eda.ipynb
-rw-rw-r--  1 clay clay 7.0K Dec 25 13:24 exp_fast_surface_dice.ipynb
-rw-rw-r--  1 clay clay  20K Dec 19 23:38 exp_grid_sample.ipynb
-rw-rw-r--  1 clay clay 3.6K Dec 25 16:26 exp_label_compare.ipynb
-rw-rw-r--  1 clay clay 462K Dec 15 20:59 foreground_splitter.ipynb
-rw-rw-r--  1 clay clay 1.4K Dec 25 16:44 ipykernel_launcher.py.pclprof
drwxrwxr-x  2 clay clay 4.0K Dec 25 16:13 .ipynb_checkpoints
-rw-rw-r--  1 clay clay 993M Dec 23 18:22 out.bin
-rw-rw-r--  1 clay clay 6.2M Dec 23 18:05 test-augmentations.ipynb
-rw-rw-r--  1 clay clay 3.2K Dec 16 14:17 unet.ipynb


In [8]:
# unique_labels = np.unique(labels_out)

In [9]:
from tqdm import tqdm


out_mmap = create_mmap_array(
    Path(f"/home/clay/research/kaggle/sennet/data_dumps/predicted/ensembled_cc3d/{folder}/chunk_00/thresholded_prob/"),
    mmap.shape,
    bool,
)
# out_mmap.data[:] = out_image

# for label, image in tqdm(cc3d.each(labels_out, binary=True, in_place=True), total=len(unique_labels)):
for label, image in tqdm(cc3d.each(labels_out, binary=True, in_place=True)):
    out_mmap.data[image] = True
    # out_image = out_image | image
    # out_image = out_image | (image > 0)
out_mmap.data.flush()


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 164/164 [01:24<00:00,  1.93it/s]


In [10]:
from sennet.core.submission_utils import generate_submission_df_from_one_chunked_inference

In [11]:
pred_dir = Path(f"/home/clay/research/kaggle/sennet/data_dumps/predicted/ensembled_cc3d/{folder}")
sub_df = generate_submission_df_from_one_chunked_inference(pred_dir)
sub_df.to_csv(pred_dir / "submission.csv")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.78s/it]


In [ ]:
# image

In [ ]:
out_mmap.data.sum() / mmap.sum()